# Load model and modules

In [4]:
from __future__ import print_function
import cobra.test
import os
from os.path import join

from __future__ import absolute_import

import re
from math import isinf, isnan
from os.path import isfile
from warnings import warn

from six import iteritems

from cobra.core import Metabolite, Model, Reaction
from cobra.util.solver import set_objective

try:
    import libsbml
except ImportError:
    libsbml = None

In [2]:
# Source: https://cobrapy.readthedocs.io/en/latest/_modules/cobra/io/sbml.html#write_cobra_model_to_sbml_file
def parse_legacy_sbml_notes(note_string, note_delimiter=':'):
    """Deal with various legacy SBML format issues.
    """
    note_dict = {}
    start_tag = '<p>'
    end_tag = '</p>'
    if '<html:p>' in note_string:
        start_tag = '<html:p>'
        end_tag = '</html:p>'
    while start_tag in note_string and end_tag in note_string:
        note_start = note_string.index(start_tag)
        note_end = note_string.index(end_tag)
        the_note = note_string[
                   (note_start + len(start_tag)):note_end].lstrip(' ').rstrip(
            ' ')
        if note_delimiter in the_note:
            note_delimiter_index = the_note.index(note_delimiter)
            note_field = the_note[:note_delimiter_index].lstrip(
                ' ').rstrip(' ').replace('_', ' ').upper()
            note_value = the_note[
                         (note_delimiter_index + 1):].lstrip(' ').rstrip(' ')
            if note_field in note_dict:
                note_dict[note_field].append(note_value)
            else:
                note_dict[note_field] = [note_value]
        note_string = note_string[(note_end + len(end_tag)):]

    if ('CHARGE' in note_dict and
            note_dict['CHARGE'][0].lower() in ['none', 'na', 'nan']):
        note_dict.pop('CHARGE')  # Remove non-numeric charges

    if 'CHARGE' in note_dict and note_dict['CHARGE'][0].lower() in ['none',
                                                                    'na',
                                                                    'nan']:
        note_dict.pop('CHARGE')  # Remove non-numeric charges

    return note_dict

def parse_legacy_id(the_id, the_compartment=None, the_type='metabolite',
                    use_hyphens=False):
    """Deals with a bunch of problems due to bigg.ucsd.edu not following SBML
    standards

    Parameters
    ----------
    the_id: String.
    the_compartment: String
    the_type: String
        Currently only 'metabolite' is supported
    use_hyphens: Boolean
        If True, double underscores (__) in an SBML ID will be converted to
        hyphens

    Returns
    -------
    string: the identifier
    """
    if use_hyphens:
        the_id = the_id.replace('__', '-')
    if the_type == 'metabolite':
        if the_id.split('_')[-1] == the_compartment:
            # Reformat Ids to match convention in Palsson Lab.
            the_id = the_id[:-len(the_compartment) - 1]
        the_id += '[%s]' % the_compartment
    return the_id

def create_cobra_model_from_sbml_file(sbml_filename, old_sbml=False,
                                      legacy_metabolite=False,
                                      print_time=False, use_hyphens=False):
    """convert an SBML XML file into a cobra.Model object.

    Supports SBML Level 2 Versions 1 and 4.  The function will detect if the
    SBML fbc package is used in the file and run the converter if the fbc
    package is used.

    Parameters
    ----------
    sbml_filename: string
    old_sbml: bool
        Set to True if the XML file has metabolite formula appended to
        metabolite names. This was a poorly designed artifact that persists in
        some models.
    legacy_metabolite: bool
        If True then assume that the metabolite id has the compartment id
         appended after an underscore (e.g. _c for cytosol). This has not been
         implemented but will be soon.
    print_time: bool
         deprecated
    use_hyphens: bool
        If True, double underscores (__) in an SBML ID will be converted to
        hyphens

    Returns
    -------
    Model : The parsed cobra model
    """
    if not libsbml:
        raise ImportError('create_cobra_model_from_sbml_file '
                          'requires python-libsbml')

    __default_lower_bound = -1000
    __default_upper_bound = 1000
    __default_objective_coefficient = 0
    # Ensure that the file exists
    if not isfile(sbml_filename):
        raise IOError('Your SBML file is not found: %s' % sbml_filename)
    # Expressions to change SBML Ids to Palsson Lab Ids
    metabolite_re = re.compile('^M_')
    reaction_re = re.compile('^R_')
    compartment_re = re.compile('^C_')
    if print_time:
        warn("print_time is deprecated", DeprecationWarning)
    model_doc = libsbml.readSBML(sbml_filename)
    if model_doc.getPlugin("fbc") is not None:
        from libsbml import ConversionProperties, LIBSBML_OPERATION_SUCCESS
        conversion_properties = ConversionProperties()
        conversion_properties.addOption(
            "convert fbc to cobra", True, "Convert FBC model to Cobra model")
        result = model_doc.convert(conversion_properties)
        if result != LIBSBML_OPERATION_SUCCESS:
            raise Exception("Conversion of SBML+fbc to COBRA failed")
    sbml_model = model_doc.getModel()
    sbml_model_id = sbml_model.getId()
    sbml_species = sbml_model.getListOfSpecies()
    sbml_reactions = sbml_model.getListOfReactions()
    sbml_compartments = sbml_model.getListOfCompartments()
    compartment_dict = dict([(compartment_re.split(x.getId())[-1], x.getName())
                             for x in sbml_compartments])
    if legacy_metabolite:
        # Deal with the palsson lab appending the compartment id to the
        # metabolite id
        new_dict = {}
        for the_id, the_name in compartment_dict.items():
            if the_name == '':
                new_dict[the_id[0].lower()] = the_id
            else:
                new_dict[the_id] = the_name
        compartment_dict = new_dict
        legacy_compartment_converter = dict(
            [(v, k) for k, v in iteritems(compartment_dict)])

    cobra_model = Model(sbml_model_id)
    metabolites = []
    metabolite_dict = {}
    # Convert sbml_metabolites to cobra.Metabolites
    for sbml_metabolite in sbml_species:
        # Skip sbml boundary species
        if sbml_metabolite.getBoundaryCondition():
            continue

        if (old_sbml or legacy_metabolite) and \
                sbml_metabolite.getId().endswith('_b'):
            # Deal with incorrect sbml from bigg.ucsd.edu
            continue
        tmp_metabolite = Metabolite()
        metabolite_id = tmp_metabolite.id = sbml_metabolite.getId()
        tmp_metabolite.compartment = compartment_re.split(
            sbml_metabolite.getCompartment())[-1]
        if legacy_metabolite:
            if tmp_metabolite.compartment not in compartment_dict:
                tmp_metabolite.compartment = legacy_compartment_converter[
                    tmp_metabolite.compartment]
            tmp_metabolite.id = parse_legacy_id(
                tmp_metabolite.id, tmp_metabolite.compartment,
                use_hyphens=use_hyphens)
        if use_hyphens:
            tmp_metabolite.id = metabolite_re.split(
                tmp_metabolite.id)[-1].replace('__', '-')
        else:
            # Just in case the SBML ids are ill-formed and use -
            tmp_metabolite.id = metabolite_re.split(
                tmp_metabolite.id)[-1].replace('-', '__')
        tmp_metabolite.name = sbml_metabolite.getName()
        tmp_formula = ''
        tmp_metabolite.notes = parse_legacy_sbml_notes(
            sbml_metabolite.getNotesString())
        if sbml_metabolite.isSetCharge():
            tmp_metabolite.charge = sbml_metabolite.getCharge()
        if "CHARGE" in tmp_metabolite.notes:
            note_charge = tmp_metabolite.notes["CHARGE"][0]
            try:
                note_charge = float(note_charge)
                if note_charge == int(note_charge):
                    note_charge = int(note_charge)
            except:
                warn("charge of %s is not a number (%s)" %
                     (tmp_metabolite.id, str(note_charge)))
            else:
                if ((tmp_metabolite.charge is None) or
                        (tmp_metabolite.charge == note_charge)):
                    tmp_metabolite.notes.pop("CHARGE")
                    # set charge to the one from notes if not assigend before
                    # the same
                    tmp_metabolite.charge = note_charge
                else:  # tmp_metabolite.charge != note_charge
                    msg = "different charges specified for %s (%d and %d)"
                    msg = msg % (tmp_metabolite.id,
                                 tmp_metabolite.charge, note_charge)
                    warn(msg)
                    # Chances are a 0 note charge was written by mistake. We
                    # will default to the note_charge in this case.
                    if tmp_metabolite.charge == 0:
                        tmp_metabolite.charge = note_charge

        for the_key in tmp_metabolite.notes.keys():
            if the_key.lower() == 'formula':
                tmp_formula = tmp_metabolite.notes.pop(the_key)[0]
                break
        if tmp_formula == '' and old_sbml:
            tmp_formula = tmp_metabolite.name.split('_')[-1]
            tmp_metabolite.name = tmp_metabolite.name[:-len(tmp_formula) - 1]
        tmp_metabolite.formula = tmp_formula
        metabolite_dict.update({metabolite_id: tmp_metabolite})
        metabolites.append(tmp_metabolite)
    cobra_model.add_metabolites(metabolites)

    # Construct the vectors and matrices for holding connectivity and numerical
    # info to feed to the cobra toolbox.
    # Always assume steady state simulations so b is set to 0
    cobra_reaction_list = []
    coefficients = {}
    for sbml_reaction in sbml_reactions:
        if use_hyphens:
            # Change the ids to match conventions used by the Palsson lab.
            reaction = Reaction(reaction_re.split(
                sbml_reaction.getId())[-1].replace('__', '-'))
        else:
            # Just in case the SBML ids are ill-formed and use -
            reaction = Reaction(reaction_re.split(
                sbml_reaction.getId())[-1].replace('-', '__'))
        cobra_reaction_list.append(reaction)
        # reaction.exchange_reaction = 0
        reaction.name = sbml_reaction.getName()
        cobra_metabolites = {}
        # Use the cobra.Metabolite class here
        for sbml_metabolite in sbml_reaction.getListOfReactants():
            tmp_metabolite_id = sbml_metabolite.getSpecies()
            # This deals with boundary metabolites
            if tmp_metabolite_id in metabolite_dict:
                tmp_metabolite = metabolite_dict[tmp_metabolite_id]
                cobra_metabolites[tmp_metabolite] = - \
                    sbml_metabolite.getStoichiometry()
        for sbml_metabolite in sbml_reaction.getListOfProducts():
            tmp_metabolite_id = sbml_metabolite.getSpecies()
            # This deals with boundary metabolites
            if tmp_metabolite_id in metabolite_dict:
                tmp_metabolite = metabolite_dict[tmp_metabolite_id]
                # Handle the case where the metabolite was specified both
                # as a reactant and as a product.
                if tmp_metabolite in cobra_metabolites:
                    warn("%s appears as a reactant and product %s" %
                         (tmp_metabolite_id, reaction.id))
                    cobra_metabolites[
                        tmp_metabolite] += sbml_metabolite.getStoichiometry()
                    # if the combined stoichiometry is 0, remove the metabolite
                    if cobra_metabolites[tmp_metabolite] == 0:
                        cobra_metabolites.pop(tmp_metabolite)
                else:
                    cobra_metabolites[
                        tmp_metabolite] = sbml_metabolite.getStoichiometry()
        # check for nan
        for met, v in iteritems(cobra_metabolites):
            if isnan(v) or isinf(v):
                warn("invalid value %s for metabolite '%s' in reaction '%s'" %
                     (str(v), met.id, reaction.id))
        reaction.add_metabolites(cobra_metabolites)
        # Parse the kinetic law info here.
        parameter_dict = {}
        # If lower and upper bounds are specified in the Kinetic Law then
        # they override the sbml reversible attribute.  If they are not
        # specified then the bounds are determined by getReversible.
        if not sbml_reaction.getKineticLaw():

            if sbml_reaction.getReversible():
                parameter_dict['lower_bound'] = __default_lower_bound
                parameter_dict['upper_bound'] = __default_upper_bound
            else:
                # Assume that irreversible reactions only proceed from left to
                # right.
                parameter_dict['lower_bound'] = 0
                parameter_dict['upper_bound'] = __default_upper_bound

            parameter_dict[
                'objective_coefficient'] = __default_objective_coefficient
        else:
            for sbml_parameter in \
                    sbml_reaction.getKineticLaw().getListOfParameters():
                parameter_dict[
                    sbml_parameter.getId().lower()] = sbml_parameter.getValue()

        if 'lower_bound' in parameter_dict:
            reaction.lower_bound = parameter_dict['lower_bound']
        elif 'lower bound' in parameter_dict:
            reaction.lower_bound = parameter_dict['lower bound']
        elif sbml_reaction.getReversible():
            reaction.lower_bound = __default_lower_bound
        else:
            reaction.lower_bound = 0

        if 'upper_bound' in parameter_dict:
            reaction.upper_bound = parameter_dict['upper_bound']
        elif 'upper bound' in parameter_dict:
            reaction.upper_bound = parameter_dict['upper bound']
        else:
            reaction.upper_bound = __default_upper_bound

        objective_coefficient = parameter_dict.get(
            'objective_coefficient', parameter_dict.get(
                'objective_coefficient', __default_objective_coefficient))
        if objective_coefficient != 0:
            coefficients[reaction] = objective_coefficient

        # ensure values are not set to nan or inf
        if isnan(reaction.lower_bound) or isinf(reaction.lower_bound):
            reaction.lower_bound = __default_lower_bound
        if isnan(reaction.upper_bound) or isinf(reaction.upper_bound):
            reaction.upper_bound = __default_upper_bound

        reaction_note_dict = parse_legacy_sbml_notes(
            sbml_reaction.getNotesString())
        # Parse the reaction notes.
        # POTENTIAL BUG: DEALING WITH LEGACY 'SBML' THAT IS NOT IN A
        # STANDARD FORMAT
        # TODO: READ IN OTHER NOTES AND GIVE THEM A reaction_ prefix.
        # TODO: Make sure genes get added as objects
        if 'GENE ASSOCIATION' in reaction_note_dict:
            rule = reaction_note_dict['GENE ASSOCIATION'][0]
            try:
                rule.encode('ascii')
            except (UnicodeEncodeError, UnicodeDecodeError):
                warn("gene_reaction_rule '%s' is not ascii compliant" % rule)
            if rule.startswith("&quot;") and rule.endswith("&quot;"):
                rule = rule[6:-6]
            reaction.gene_reaction_rule = rule
            if 'GENE LIST' in reaction_note_dict:
                reaction.systematic_names = reaction_note_dict['GENE LIST'][0]
            elif ('GENES' in reaction_note_dict and
                  reaction_note_dict['GENES'] != ['']):
                reaction.systematic_names = reaction_note_dict['GENES'][0]
            elif 'LOCUS' in reaction_note_dict:
                gene_id_to_object = dict([(x.id, x) for x in reaction._genes])
                for the_row in reaction_note_dict['LOCUS']:
                    tmp_row_dict = {}
                    the_row = 'LOCUS:' + the_row.lstrip('_').rstrip('#')
                    for the_item in the_row.split('#'):
                        k, v = the_item.split(':')
                        tmp_row_dict[k] = v
                    tmp_locus_id = tmp_row_dict['LOCUS']
                    if 'TRANSCRIPT' in tmp_row_dict:
                        tmp_locus_id = tmp_locus_id + \
                                       '.' + tmp_row_dict['TRANSCRIPT']

                    if 'ABBREVIATION' in tmp_row_dict:
                        gene_id_to_object[tmp_locus_id].name = tmp_row_dict[
                            'ABBREVIATION']

        if 'SUBSYSTEM' in reaction_note_dict:
            reaction.subsystem = reaction_note_dict.pop('SUBSYSTEM')[0]

        reaction.notes = reaction_note_dict

    # Now, add all of the reactions to the model.
    cobra_model.id = sbml_model.getId()
    # Populate the compartment list - This will be done based on
    # cobra.Metabolites in cobra.Reactions in the future.
    cobra_model.compartments = compartment_dict

    cobra_model.add_reactions(cobra_reaction_list)
    set_objective(cobra_model, coefficients)
    return cobra_model

In [5]:
# Loading the original model
MODEL = create_cobra_model_from_sbml_file('Paracoccus_autoTemplate_completeMedia.sbml', old_sbml=False, legacy_metabolite=False, print_time=False, use_hyphens=False)

# Add general (de)nitrification reactions

In [6]:
# Add missing transport and exchange reactions nitrogen metabolism
# (denitrification and heterotrophic nitrification)

# Nitrogen diffusion: N2_c0 <=> N2_e0
# cpd00528[1] <=> cpd00528
rxn10577_c0 = Reaction('rxn10577_c0')
rxn10577_c0.name = 'Nitrogen exchange, diffusion'
rxn10577_c0.subsystem = 'Transport'
rxn10577_c0.lower_bound = -1000.0
rxn10577_c0.upper_bound = 1000.0

cpd00528_c0 = MODEL.metabolites.get_by_id('cpd00528_c0')
cpd00528_e0 = Metabolite(
    'cpd00528_e0', 
    formula = 'N2', 
    name = 'N2_e',
    compartment = 'e0')

rxn10577_c0.add_metabolites({
    cpd00528_c0: -1.0,
    cpd00528_e0: 1.0
})

# Nitrite transport :NO2_e0 <=> NO2_c0
# cpd00075 <=> cpd00075
rxn08999_c0 = Reaction('rxn08999_c0')
rxn08999_c0.name = 'nitrite transport via diffusion (extracellular to periplasm)'
rxn08999_c0.subsystem = 'Transport'
rxn08999_c0.lower_bound = -1000.0
rxn08999_c0.upper_bound = 1000.0

cpd00075_c0 = MODEL.metabolites.get_by_id('cpd00075_c0')
cpd00075_e0 =  Metabolite(
    'cpd00075_e0', 
    formula = 'NO2', 
    name = 'NO2_e',
    compartment = 'e0')

rxn08999_c0.add_metabolites({
    cpd00075_c0: 1.0,
    cpd00075_e0: -1.0
})

# Nitrate transport :NO3_e0 <=> NO3_c0
# cpd00209 <=> cpd00209
rxn09005_c0 = Reaction('rxn09005_c0')
rxn09005_c0.name = 'nitrate transport via diffusion (extracellular to periplasm)'
rxn09005_c0.subsystem = 'Transport'
rxn09005_c0.lower_bound = -1000.0
rxn09005_c0.upper_bound = 1000.0

cpd00209_c0 = MODEL.metabolites.get_by_id('cpd00209_c0')
cpd00209_e0 =  Metabolite(
    'cpd00209_e0', 
    formula = 'NO3', 
    name = 'NO3_e',
    compartment = 'e0')

rxn09005_c0.add_metabolites({
    cpd00209_c0: 1.0,
    cpd00209_e0: -1.0
})

#add boundary reactions for dinitrogen and nitrite
MODEL.add_boundary(cpd00528_e0, type="exchange", reaction_id="EX_cpd00528_e0", ub=1000.)
MODEL.add_boundary(cpd00075_e0, type="exchange", reaction_id="EX_cpd00075_e0", ub=1000.)
MODEL.add_boundary(cpd00209_e0, type="exchange", reaction_id="EX_cpd00209_e0", ub=1000.)

MODEL.add_reactions([rxn10577_c0, rxn08999_c0, rxn09005_c0])

In [7]:
# Fix oxidative phosphorylation
# (2) Cytochrome c3+[1] + Ubiquinol-8[1] <=> (2) H+ + (2) Cytochrome c2+[1] + Ubiquinone-8[1]
# (2) cpd00109 + cpd15561 <=> (2) cpd00067 + (2) cpd00110 + cpd15560
rxn12750_c0 = Reaction('rxn12750_c0')
rxn12750_c0.name = 'ubiquinol---cytochrome-c reductase'
rxn12750_c0.subsystem = 'Oxidative phosphorylation'
rxn12750_c0.lower_bound = -1000.0
rxn12750_c0.upper_bound = 1000.0

cpd00109_c0 = MODEL.metabolites.get_by_id('cpd00109_c0')
cpd15561_c0 = MODEL.metabolites.get_by_id('cpd15561_c0')
cpd00067_c0 = MODEL.metabolites.get_by_id('cpd00067_c0')
cpd00110_c0 = MODEL.metabolites.get_by_id('cpd00110_c0')
cpd15560_c0 = MODEL.metabolites.get_by_id('cpd15560_c0')

rxn12750_c0.add_metabolites({
    cpd00109_c0: -2.0,
    cpd15561_c0: -1.0,
    cpd00067_c0: 2.0,
    cpd00110_c0: 2.0,
    cpd15560_c0: 1.0
})

MODEL.add_reactions([rxn12750_c0])

In [8]:
# Fix denitrification pathway

# knock-out menaquinol dependent nitrate reductase
MODEL. reactions.rxn10121_c0.knock_out()

# H2O + Cytochrome c3+ + NO <= H+ + Nitrite + Cytochrome c2+
# cpd00001 + cpd00109 + cpd00418 <= cpd00067 + cpd00075 + cpd00110
rxn00567_c0 = Reaction('rxn00567_c0')
rxn00567_c0.name = 'nitric-oxide:ferricytochrome-c oxidoreductase'
rxn00567_c0.subsystem = 'Denitrification'
rxn00567_c0.lower_bound = -1000.0
rxn00567_c0.upper_bound = 0.0

cpd00001_c0 = MODEL.metabolites.get_by_id('cpd00001_c0')
cpd00109_c0 = MODEL.metabolites.get_by_id('cpd00109_c0')
cpd00418_c0 = MODEL.metabolites.get_by_id('cpd00418_c0')
cpd00067_c0 = MODEL.metabolites.get_by_id('cpd00067_c0')
cpd00075_c0 = MODEL.metabolites.get_by_id('cpd00075_c0')
cpd00110_c0 = MODEL.metabolites.get_by_id('cpd00110_c0')

rxn00567_c0.add_metabolites({
    cpd00001_c0: -1.0,
    cpd00109_c0: -1.0,
    cpd00418_c0: -1.0,
    cpd00067_c0: 1.0,
    cpd00075_c0: 1.0,
    cpd00110_c0: 1.0
})

# knock-out nitrous_oxide_NAD_plus__oxidoreductase
MODEL. reactions.rxn01806_c0.knock_out()

# (2) NO + Cytochrome_c2_plus <=> H2O + Nitrous oxide + Cytochrome_c3_plus
# (2) cpd00418 + cpd00110 <=> cpd00001 + cpd00659 + cpd00109
rxn05801mod_c0 = Reaction('rxn05801mod_c0')
rxn05801mod_c0.name = 'nitrous-oxide:ferricytochrome-c oxidoreductase'
rxn05801mod_c0.subsystem = 'Denitrification'
rxn05801mod_c0.lower_bound = -1000.0
rxn05801mod_c0.upper_bound = 1000.0

cpd00110_c0 = MODEL.metabolites.get_by_id('cpd00110_c0')
cpd00659_c0 = MODEL.metabolites.get_by_id('cpd00659_c0')
cpd00109_c0 = MODEL.metabolites.get_by_id('cpd00109_c0')

rxn05801mod_c0.add_metabolites({
    cpd00418_c0: -2.0,
    cpd00110_c0: -1.0,
    cpd00001_c0: 1.0,
    cpd00001_c0: 1.0,
    cpd00109_c0: 1.0
})

# knock-out Nitrogen_NAD_oxidoreductase_N2O_forming
MODEL. reactions.rxn11937_c0.knock_out()

# H2O + (2) Cytochrome c3+ + N2 <=> (2) H+ + (2) Cytochrome c2+ + Nitrous oxide
# cpd00001 + (2) cpd00109 + cpd00528 <=> (2) cpd00067 + (2) cpd00110 + cpd00659
rxn39362_c0 = Reaction('rxn39362_c0')
rxn39362_c0.name = 'nitrogen:ferricytochrome-c oxidoreductase (N2O-forming)'
rxn39362_c0.subsystem = 'Denitrification'
rxn39362_c0.lower_bound = -1000.0
rxn39362_c0.upper_bound = 0.0

cpd00528_c0 = MODEL.metabolites.get_by_id('cpd00528_c0')

rxn39362_c0.add_metabolites({
    cpd00001_c0: -1.0,
    cpd00109_c0: -2.0,
    cpd00528_c0: -1.0,
    cpd00067_c0: 1.0,
    cpd00110_c0: 2.0,
    cpd00659_c0: 1.0
})

MODEL.add_reactions([rxn00567_c0, rxn05801mod_c0, rxn39362_c0])

# Add heterotrophic nitrification-aerobic denitrification in seperate reactions

In [ ]:
# Fix/add heterotrophic nitrification pathway >> adds seperate reactions (lumped below)
seperate = False

if seperate == True:

    # O2 + NH3 + ubiquinol-8 => H2O + H+ + Hydroxylamine + ubiquinone-8
    # cpd00007 + cpd00013 + cpd15561 => cpd00001 + cpd00067 + cpd00165 + cpd15560
    rxn14010mod_c0 = Reaction('rxn14010_c0')
    rxn14010mod_c0.name = 'ammonia,ubiquinol:oxygen oxidoreductase'
    rxn14010mod_c0.subsystem = 'heterotrophic nitrification'
    rxn14010mod_c0.lower_bound = 0.0
    rxn14010mod_c0.upper_bound = 1000.0

    cpd00007_c0 = MODEL.metabolites.get_by_id('cpd00007_c0')
    cpd00013_c0 = MODEL.metabolites.get_by_id('cpd00013_c0')
    cpd15561_c0 = MODEL.metabolites.get_by_id('cpd15561_c0')
    cpd00001_c0 = MODEL.metabolites.get_by_id('cpd00001_c0')
    cpd00067_c0 = MODEL.metabolites.get_by_id('cpd00067_c0')
    cpd00165_c0 = Metabolite(
        'cpd00165_c0',
        formula='H3NO',
        name='Hydroxylamine_c0',
        compartment='c0')
    cpd15560_c0 = MODEL.metabolites.get_by_id('cpd15560_c0')

    rxn14010mod_c0.add_metabolites({
        cpd00007_c0: -1.0,
        cpd00013_c0: -1.0,
        cpd15561_c0: -1.0,
        cpd00001_c0: 1.0,
        cpd00067_c0: 1.0,
        cpd00165_c0: 1.0,
        cpd15560_c0: 1.0
    })

    # Cytochrome c551+ as cpd18072_c0 (Cytochrome C-ox) & cytochrome c551 as cpd18074_c0 (Cytochrome C-rex)
    # H2O + (4) Cytochrome c551+ + Hydroxylamine <=> Nitrite + (4) Cytochrome c551
    # cpd00001 + (4) cpd18072 + cpd00165 <=> cpd00075 + (4) cpd18074
    rxn40053mod_c0 = Reaction('rxn40053mod_c0')
    rxn40053mod_c0.name = 'hydroxylamine:ferricytochrome-c oxidoreductase'
    rxn40053mod_c0.subsystem = 'heterotrophic nitrification'
    rxn40053mod_c0.lower_bound = -1000.0 
    rxn40053mod_c0.upper_bound = 1000.0

    cpd18072_c0 = Metabolite(
        'cpd18072_c0',
        formula='Unkown',
        name='Cytochrome 551+ or C-ox',
        compartment='c0')
    cpd00075_c0 = MODEL.metabolites.get_by_id('cpd00075_c0')
    cpd18074_c0 = Metabolite(
        'cpd18074_c0',
        formula='Unkown',
        name='Cytochrome 551 or C-red',
        compartment='c0')


    rxn40053mod_c0.add_metabolites({
        cpd00001_c0: -1.0,
        cpd18072_c0: -4.0,
        cpd00165_c0: -1.0,
        cpd00075_c0: 1.0,
        cpd18074_c0: 4.0
    })

    # H2O + NO + Cytochrome 551+ <=> H+ + Nitrite + Cytochrome 551
    # cpd00001 + cpd00418 + cpd18072 <=> cpd00067 + cpd00075 + cpd18074
    rxn14428mod_c0 = Reaction('rxn14428mod_c0')
    rxn14428mod_c0.name = 'Nitrite reductase cytochrome-c type'
    rxn14428mod_c0.subsystem = 'heterotrophic nitrification'
    rxn14428mod_c0.lower_bound = -1000.0 
    rxn14428mod_c0.upper_bound = 1000.0

    cpd00418_c0 = MODEL.metabolites.get_by_id('cpd00418_c0')

    rxn14428mod_c0.add_metabolites({
        cpd00001_c0: -1.0,
        cpd00418_c0: -1.0,
        cpd18072_c0: -1.0,
        cpd00067_c0: 1.0,
        cpd00075_c0: 1.0,
        cpd18074_c0: 1.0
    })

    # H2O + Nitrous oxide + (2) Cytochromes c551+ <=> (2) H+ + (2) NO + (2) Cytochrome c551
    # cpd00001 + cpd00659 + (2) cpd18072 <=> (2) cpd00067 + (2) cpd00418 + (2) cpd18074
    rxn20282mod_c0 = Reaction('rxn20282mod_c0')
    rxn20282mod_c0.name = 'nitrogen oxide reductase'
    rxn20282mod_c0.subsystem = 'heterotrophic nitrification'
    rxn20282mod_c0.lower_bound = -1000.0 
    rxn20282mod_c0.upper_bound = 1000.0

    cpd00659_c0 = MODEL.metabolites.get_by_id('cpd00659_c0')

    rxn20282mod_c0.add_metabolites({
        cpd00001_c0: -1.0,
        cpd00659_c0: -1.0,
        cpd18072_c0: -2.0,
        cpd00067_c0: 2.0,
        cpd00418_c0: 2.0,
        cpd18074_c0: 2.0
    })

    # H2O + N2 + (2) Cytochromes c551+ <= (2) H+ + Nitrous oxide + (2) Cytochromes c551
    # cpd00001 + cpd00528 + (2) cpd18072 <= (2) cpd00067 + cpd00659 + (2) cpd18074
    rxn22446mod_c0 = Reaction('rxn22446mod_c0')
    rxn22446mod_c0.name = 'nitrous-oxide reductase'
    rxn22446mod_c0.subsystem = 'heterotrophic nitrification'
    rxn22446mod_c0.lower_bound = -1000.0 
    rxn22446mod_c0.upper_bound = 0.0

    cpd00528_c0 = MODEL.metabolites.get_by_id('cpd00528_c0')

    rxn22446mod_c0.add_metabolites({
        cpd00001_c0: -1.0,
        cpd00528_c0: -1.0,
        cpd18072_c0: -2.0,
        cpd00067_c0: 2.0,
        cpd00659_c0: 1.0,
        cpd18074_c0: 2.0
    })

    # (0.5) O2 + (4) H+ + Cytochrome c551 <=> H2O + (2) H+[1] + Cytochrome c551+
    # (0.5) cpd00007 + (4) cpd00067 + cpd18074 <=> cpd00001 + (4) cpd00067[1] + cpd18072
    rxn10043mod_c0 = Reaction('rxn10043mod_c0')
    rxn10043mod_c0.name = 'cytochrome-c oxidase'
    rxn10043mod_c0.subsystem = 'heterotrophic nitrification'
    rxn10043mod_c0.lower_bound = -1000.0 
    rxn10043mod_c0.upper_bound = 1000.0

    cpd00067_e0 = MODEL.metabolites.get_by_id('cpd00067_e0')

    rxn10043mod_c0.add_metabolites({
        cpd00007_c0: -0.5,
        cpd00067_c0: -4.0,
        cpd18074_c0: -1.0,
        cpd00001_c0: 1.0,
        cpd00067_e0: 1.0,
        cpd18072_c0: 1.0
    })

    MODEL.add_reactions([rxn14010mod_c0, rxn40053mod_c0, rxn14428mod_c0, rxn20282mod_c0, rxn22446mod_c0, rxn10043mod_c0])
    
    MODEL.repair()

In [ ]:
# Save the model with seperated (de)nitrification reactions
seperateSave = False

if seperateSave == True:

    cobra.io.write_sbml_model(MODEL, "Paracoccus_seperateNitrification_completeMedia.sbml")

# Add heterotrophic nitrification-aerobic denitrification in lumped reactions

In [9]:
# Fix/add heterotrophic nitrification pathway >> adds lumped reactions
lumped = False

if lumped == True:
    
    # O2 + NH3 + ubiquinol-8 => H2O + H+ + Hydroxylamine + ubiquinone-8
    # cpd00007 + cpd00013 + cpd15561 => cpd00001 + cpd00067 + cpd00165 + cpd15560
    rxn14010mod_c0 = Reaction('rxn14010_c0')
    rxn14010mod_c0.name = 'ammonia,ubiquinol:oxygen oxidoreductase'
    rxn14010mod_c0.subsystem = 'heterotrophic nitrification'
    rxn14010mod_c0.lower_bound = 0.0
    rxn14010mod_c0.upper_bound = 1000.0

    cpd00007_c0 = MODEL.metabolites.get_by_id('cpd00007_c0')
    cpd00013_c0 = MODEL.metabolites.get_by_id('cpd00013_c0')
    cpd15561_c0 = MODEL.metabolites.get_by_id('cpd15561_c0')
    cpd00001_c0 = MODEL.metabolites.get_by_id('cpd00001_c0')
    cpd00067_c0 = MODEL.metabolites.get_by_id('cpd00067_c0')
    cpd00165_c0 = Metabolite(
        'cpd00165_c0',
        formula='H3NO',
        name='Hydroxylamine_c0',
        compartment='c0')
    cpd15560_c0 = MODEL.metabolites.get_by_id('cpd15560_c0')

    rxn14010mod_c0.add_metabolites({
        cpd00007_c0: -1.0,
        cpd00013_c0: -1.0,
        cpd15561_c0: -1.0,
        cpd00001_c0: 1.0,
        cpd00067_c0: 1.0,
        cpd00165_c0: 1.0,
        cpd15560_c0: 1.0
    })

    # Cytochrome c551+ as cpd18072_c0 (Cytochrome C-ox) & cytochrome c551 as cpd18074_c0 (Cytochrome C-rex)
    # H2O + (4) Cytochrome c551+ + Hydroxylamine <=> Nitrite + (4) Cytochrome c551
    # cpd00001 + (4) cpd18072 + cpd00165 <=> cpd00075 + (4) cpd18074
    rxn40053mod_c0 = Reaction('rxn40053mod_c0')
    rxn40053mod_c0.name = 'hydroxylamine:ferricytochrome-c oxidoreductase'
    rxn40053mod_c0.subsystem = 'heterotrophic nitrification'
    rxn40053mod_c0.lower_bound = -1000.0 
    rxn40053mod_c0.upper_bound = 1000.0

    cpd18072_c0 = Metabolite(
        'cpd18072_c0',
        formula='Unkown',
        name='Cytochrome 551+ or C-ox',
        compartment='c0')
    cpd00075_c0 = MODEL.metabolites.get_by_id('cpd00075_c0')
    cpd18074_c0 = Metabolite(
        'cpd18074_c0',
        formula='Unkown',
        name='Cytochrome 551 or C-red',
        compartment='c0')


    rxn40053mod_c0.add_metabolites({
        cpd00001_c0: -1.0,
        cpd18072_c0: -4.0,
        cpd00165_c0: -1.0,
        cpd00075_c0: 1.0,
        cpd18074_c0: 4.0
    })

    # (7) H+ + Nitrite + (0.5) O2 + (4) Cytochrome c551 => (3) H2O + (0.5) N2 + (2) H+[e] + (4) Cytochrome c551+
    # (7) cpd00067 + cpd00075 + (0.5) cpd00007 + (4) cpd18074 => (3) cpd00001 + (0.5) cpd00528 + (2) cpd00067_e0 + (4) cpd18072
    rxn10043lump_c0 = Reaction('rxn10043lump_c0')
    rxn10043lump_c0.name = 'cytochrome-c oxidase'
    rxn10043lump_c0.subsystem = 'heterotrophic nitrification'
    rxn10043lump_c0.lower_bound = 0.0 
    rxn10043lump_c0.upper_bound = 1000.0

    cpd00528_c0 = MODEL.metabolites.get_by_id('cpd00528_c0')
    cpd00067_e0 = MODEL.metabolites.get_by_id('cpd00067_e0')

    rxn10043lump_c0.add_metabolites({
        cpd00067_c0: -7,
        cpd00075_c0: -1.0,
        cpd00007_c0: -0.5,
        cpd18074_c0: -4.0,
        cpd00001_c0: 3.0,
        cpd00528_c0: 0.5,
        cpd00067_e0: 2.0,
        cpd18072_c0: 4.0
    })

    MODEL.add_reactions([rxn14010mod_c0, rxn40053mod_c0, rxn10043lump_c0])
    
    MODEL.repair()

In [12]:
# Save the adjusted model
lumped = False

if lumped == True:
    
    cobra.io.write_sbml_model(MODEL, "Paracoccus_lumpedNitrification_completeMedia.sbml")